In [1]:
import pandas as pd
from scipy.stats import pearsonr
from scipy.stats import chisquare

# 编码相关性

In [3]:
df = pd.read_excel("编码分月占比.xlsx",index_col=0)
df.head()

,公共事件,娱乐体育,新冠疫情
月份,,,
2020-01,0.507289,0.285714,0.370262
2020-02,0.601719,0.232092,0.484241
2020-03,0.500000,0.274566,0.384393
2020-04,0.432277,0.279539,0.285303
2020-05,0.334311,0.296188,0.090909


In [18]:
corr1 = []
corr1.append(pearsonr(df['公共事件'],df['娱乐体育'])[0])
for i in range(4):
    corr1.append(pearsonr(df['公共事件'][i*6:(i+1)*6],df['娱乐体育'][i*6:(i+1)*6])[0])
    
corr2 = []
corr2.append(pearsonr(df['公共事件'],df['新冠疫情'])[0])
for i in range(4):
    corr2.append(pearsonr(df['公共事件'][i*6:(i+1)*6],df['新冠疫情'][i*6:(i+1)*6])[0])
    
corr3 = []
corr3.append(pearsonr(df['娱乐体育'],df['新冠疫情'])[0])
for i in range(4):
    corr3.append(pearsonr(df['娱乐体育'][i*6:(i+1)*6],df['新冠疫情'][i*6:(i+1)*6])[0])

In [22]:
result1 = pd.DataFrame([corr1,corr2,corr3],
                      index=['公共事件与娱乐体育','公共事件与新冠疫情','娱乐体育与新冠疫情'],
                       columns= ['总计','2020H1','2020H2','2021H1','2021H2'])

result1.to_excel("编码相关系数.xlsx")
result1.head()

,总计,2020H1,2020H2,2021H1,2021H2
公共事件与娱乐体育,-0.523097,-0.709846,-0.917313,-0.360016,-0.272829
公共事件与新冠疫情,0.454156,0.987531,-0.078062,0.107425,-0.077066
娱乐体育与新冠疫情,-0.652411,-0.713250,-0.166786,-0.180364,0.298964


# 卡方检验

In [2]:
def classify(x):
    if x<=16:
        return '1-16'
    elif x >= 17 and x<=32:
        return '17-32'
    elif x >=33:
        return '33-50'

In [3]:
df1 = pd.read_excel("编码.xlsx")
df1['月份'] = df1['日期'].apply(lambda x:x[:7])
df1['热搜排名'] = df1['排名'].apply(classify)
df1.head()

,日期,排名,热搜词,热度,公共事件,新冠疫情,娱乐体育,月份,热搜排名
0,2020-01-07 16:00:00,1,郑爽爸爸微博评论,3233504,1,0,1,2020-01,1-16
1,2020-01-07 16:00:00,2,伊朗外长被美国拒签,1449909,1,0,0,2020-01,1-16
2,2020-01-07 16:00:00,3,鹿晗 我状态跟你有啥关系,1433392,0,0,1,2020-01,1-16
3,2020-01-07 16:00:00,4,现代版庆余年,1235314,0,0,1,2020-01,1-16
4,2020-01-07 16:00:00,5,宋祖儿方否认与阮经天恋情,907979,0,0,1,2020-01,1-16


In [4]:
pivot_table = pd.pivot_table(df1,
                             index=['月份'],
                              columns='热搜排名',
                              values=['公共事件','新冠疫情','娱乐体育'],
                             aggfunc='mean')

pivot_table.head()

公共事件                          娱乐体育                          新冠疫情  \
热搜排名         1-16     17-32     33-50      1-16     17-32     33-50      1-16   
月份                                                                              
2020-01  0.607143  0.482143  0.436975  0.321429  0.241071  0.294118  0.464286   
2020-02  0.678571  0.526786  0.600000  0.169643  0.285714  0.240000  0.598214   
2020-03  0.616071  0.357143  0.524590  0.267857  0.392857  0.172131  0.482143   
2020-04  0.464286  0.383929  0.447154  0.339286  0.285714  0.219512  0.285714   
2020-05  0.330357  0.196429  0.470085  0.366071  0.348214  0.179487  0.098214   

                             
热搜排名        17-32     33-50  
月份                           
2020-01  0.303571  0.344538  
2020-02  0.446429  0.416000  
2020-03  0.294643  0.377049  
2020-04  0.214286  0.349593  
2020-05  0.071429  0.102564

In [7]:
result2 = []
time_index = ['2020H1','2020H2','2021H1','2021H2']
for column1 in ['公共事件','娱乐体育','新冠疫情']:
    for i in range(4):
        p1 = chisquare(pivot_table[column1]['1-16'][6*i:6*(i+1)],pivot_table[column1]['17-32'][6*i:6*(i+1)])[0]
        p2 = chisquare(pivot_table[column1]['1-16'][6*i:6*(i+1)],pivot_table[column1]['33-50'][6*i:6*(i+1)])[0]
        p3 = chisquare(pivot_table[column1]['17-32'][6*i:6*(i+1)],pivot_table[column1]['33-50'][6*i:6*(i+1)])[0]
        result2.append([column1,time_index[i],round(p1,2),round(p2,2),round(p3,2)])
    
result2 = pd.DataFrame(result2,
            columns=['编码类别','时间段','前部 vs 中部','前部 vs 后部','中部 vs 后部'])

result2.to_excel("热搜分段chisquare.xlsx",index=False)
result2

,编码类别,时间段,前部 vs 中部,前部 vs 后部,中部 vs 后部
0,公共事件,2020H1,0.47,0.14,0.28
1,公共事件,2020H2,0.04,0.07,0.02
2,公共事件,2021H1,0.13,0.03,0.09
3,公共事件,2021H2,0.49,0.15,0.17
4,娱乐体育,2020H1,0.12,0.41,0.54
5,娱乐体育,2020H2,0.10,0.13,0.01
6,娱乐体育,2021H1,0.02,0.25,0.14
7,娱乐体育,2021H2,0.24,0.19,0.22
8,新冠疫情,2020H1,0.36,0.16,0.11
9,新冠疫情,2020H2,1.62,0.17,0.11


In [45]:
chisquare(pivot_table[column1[0]]['17-32'],pivot_table[column1[0]]['33-50'])[1]

0.9999999999999999

总和

In [9]:
result3 = []
for column1 in ['公共事件','娱乐体育','新冠疫情']:
    p1 = chisquare(pivot_table[column1]['1-16'],pivot_table[column1]['17-32'])[0]
    p2 = chisquare(pivot_table[column1]['1-16'],pivot_table[column1]['33-50'])[0]
    p3 = chisquare(pivot_table[column1]['17-32'],pivot_table[column1]['33-50'])[0]
    result3.append([column1,round(p1,2),round(p2,2),round(p3,2)])
    
result3 = pd.DataFrame(result3,
            columns=['编码类别','前部 vs 中部','前部 vs 后部','中部 vs 后部'])

result3.to_excel("热搜chisquare-总和.xlsx",index=False)
result3

,编码类别,前部 vs 中部,前部 vs 后部,中部 vs 后部
0,公共事件,1.14,0.38,0.56
1,娱乐体育,0.49,0.98,0.92
2,新冠疫情,2.89,0.71,0.42
